## Importing Modules

In [ ]:
#pip install yfinance


In [ ]:
import json
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import yfinance as yahooFinance
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import *

## Data Preprocessing

### Raw Data From Yahoofinance

In [ ]:
Ticker_Of_Firm='GAIL.NS'
Information = yahooFinance.Ticker(Ticker_Of_Firm)
Name_Of_Firm=Information.info['shortName']
df=Information.history(period='max')
df.head(10)

### Graphs


In [ ]:
fig, axs = plt.subplots(3, 1, constrained_layout=True,figsize=(12,15))
fig.suptitle(Name_Of_Firm, fontsize=16)


# for Full Data Available
axs[0].plot(Information.history(period='max')[['Open']],color='lightblue')
axs[0].set_title('Maximum Period')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Price Of Stock (INR) ')

# for One Year
axs[1].plot(Information.history(period='1y')[['Open']],color='orange')
axs[1].set_xlabel('Time')
axs[1].set_title('One Year')
axs[1].set_ylabel('Price Of Stock (INR) ')

# for One Month 
axs[2].plot(Information.history('1mo')[['Open']],color='green')
axs[2].set_xlabel('Time')
axs[2].set_title('One Month')
axs[2].set_ylabel('Price Of Stock (INR) ')
#plt.show()

### Understanding Of Data

In [ ]:
#Data
Data=Information.history(period='max')[['Open']]
Data=np.array(Data)[-5000:]

scaler=MinMaxScaler(feature_range=(0, 1))
scaler=scaler.fit(Data)
Data_Normalised=scaler.transform(Data).flatten()
Data=Data.flatten()
Data_size=Data.size
Validation_size=100
Time=np.arange(1,Data_size+1)
print(f'The size of data set is: {Data_size}')
print(f'The size of training data set is: {Data_size-Validation_size}')
print(f'The size of validation data set is: {Validation_size}')
Data_Dictionary={
'Series':Data[:]  ,  
'Data_Training':Data[:Data_size-Validation_size],
'Data_Validation':Data[Data_size-Validation_size:],
'Time_Training':Time[:Data_size-Validation_size],
'Time_Validation':Time[Data_size-Validation_size:]
}
    
Data_Normalised_Dictionary={
'Series':Data_Normalised[:]  ,  
'Data_Training':Data_Normalised[:Data_size-Validation_size],
'Data_Validation':Data_Normalised[Data_size-Validation_size:],
'Time_Training':Time[:Data_size-Validation_size],
'Time_Validation':Time[Data_size-Validation_size:]
}    
Data_Normalised_Dictionary=Data_Dictionary    
#Figure
fig=plt.figure(figsize=(10,6))
plt.plot(Data_Dictionary['Time_Training'],Data_Dictionary['Data_Training'])
plt.plot(Data_Dictionary['Time_Validation'],Data_Dictionary['Data_Validation'])
plt.ylabel('Price Of Stock (INR) ')
plt.legend(['Training Set','Validation Set'],fontsize=15)
plt.show()

### Setting Of Data and Initial Parameters

In [ ]:
window_size=60
batch_size=200
shuffle_buffer=1000

# function for windowing of data set and making of batches
def windowed_dataset(data):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    dataset = dataset.window(window_size+1,shift=1,drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))        
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset


## Model Architecture

In [ ]:
#setting random seed for numpy and tensorflow
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)


# Halt Calss , Function call after every epoch
class haltCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        loss_value=1
        if(logs.get('loss') <= loss_value):
            print(f"\n\n\nReached {loss_value} loss value so cancelling training!\n\n\n")
            self.model.stop_training = True

trainingStopCallback=haltCallback()        
            
# model
model = tf.keras.models.Sequential([
          tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),input_shape=[None]),  # Lambda layer for expanding dimention
          tf.keras.layers.SimpleRNN(40,return_sequences=True),
          tf.keras.layers.SimpleRNN(40,return_sequences=False),
          #tf.keras.layers.LSTM(45,activation ='tanh'),
          tf.keras.layers.Dense(1),
          tf.keras.layers.Lambda(lambda x: x * 100.0)
        ])






# Optimization and Loss

initial_learning_rate = 4e-8
decay_steps = 207*1.2
decay_rate = 0.5
no_of_epoch=200
no_of_batch=Data_size//batch_size
learning_rate_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
                          initial_learning_rate=initial_learning_rate,
                          decay_steps=decay_steps,
                          decay_rate=decay_rate)



optimizer = tf.keras.optimizers.SGD(learning_rate_schedule,momentum=0.9)     #SGD was old optimizer, momentum was 0.9
model.compile(loss=tf.keras.losses.MeanSquaredError() ,                      # earlier it was Huber()
                      optimizer=optimizer,
                      metrics=["mae"])
# Take a look at our Model
print(model.summary())
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True )

## Decay In Learning Rate

In [ ]:
step = np.arange(0,no_of_epoch*no_of_batch)
lr = learning_rate_schedule(step)
fig=plt.figure(figsize=(10, 6))
fig.suptitle('LR vs Epoch')
plt.plot(step/no_of_batch,lr)
plt.ylabel('Learning Rate')
_=plt.xlabel('Epoch Number')

### Fitting and Forecasting

In [ ]:
tf.keras.backend.clear_session()
#tf.random.set_seed(27)
#np.random.seed(27)

dataset=windowed_dataset(Data_Normalised_Dictionary['Data_Training'])
history = model.fit(dataset,epochs=no_of_epoch,callbacks=[trainingStopCallback])

In [ ]:
#forecasting

forecast=[]
tmp=Data_Normalised_Dictionary['Series'][-(window_size+Validation_size):]
for i in range(Validation_size):
    re=model.predict(tmp[i:i+window_size][np.newaxis])
    forecast.append(re[0,0])
forecast=np.array(forecast)    
#forecast=scaler.inverse_transform(forecast.reshape((-1,1))).flatten() 

    


In [ ]:
#figure
fig=plt.figure(figsize=(10, 6))
fig.suptitle('Prediction Graph')
plt.plot(Data_Dictionary['Time_Validation'],forecast)
plt.plot(Data_Dictionary['Time_Validation'],Data_Dictionary['Data_Validation'])
plt.ylabel('Price Of Stock (INR) ')
plt.xlabel('Day Number')

plt.legend(['Predicted','Actual'],fontsize=15)
plt.show()

## Analysis